# Scavengers cha-1

In [ ]:
with open('token.txt', 'r') as f:
    acceso=f.readlines()[0].split('=')

user=acceso[0]
key=acceso[1].rstrip('\n')

In [ ]:
# https://developer.github.com/v3/guides/

In [ ]:
import json
import requests
import pandas as pd
import time

In [ ]:
repo='ironhack-datalabs/madrid-oct-2018'

get_forks='http://api.github.com/repos/'+ repo +'/forks'

In [ ]:
res_fork=requests.get(get_forks, auth=(user, key))

res=res_fork.json()

In [ ]:
res[0].keys()

In [ ]:
languages=set()

for i in range(len(res)):
    languages.add(res[i]['language'])

print(languages)

In [ ]:
lan=[]

for i in range(len(res)):
    lan.append(res[i]['language'])
    
set(lan)

# Scavengers cha-2

In [ ]:
repo='YonatanRA/clases_datamex0120'

desde='2020-01-01T00:00:00Z'
hasta='2020-01-31T23:59:59Z'

In [ ]:
get_commits='http://api.github.com/repos/{}/commits?since={}&until={}'.format(repo, desde, hasta)

res_commit=requests.get(get_commits, auth=(user, key))
results_commit=res_commit.json()

In [ ]:
commits=[]
for i in range(len(results_commit)):
    commits.append(results_commit[i]['committer'])

print(len(commits)) 

# Scavengers cha-3 V1

In [ ]:
repo='ironhack-datalabs/scavenger'

In [ ]:
get_repo='http://api.github.com/repos/{}/git/trees/master?recursive=1'.format(repo)

get_repo=requests.get(get_repo, auth=(user, key))
res=get_repo.json()
res['tree']

In [ ]:
res['tree'][7]['path']

In [ ]:
url='https://api.github.com/repos/ironhack-datalabs/scavenger/contents/'

tree=res['tree']

archivos=[]
for i in range(len(tree)):
    if tree[i]['type']=='blob' and 'scavengerhunt' in tree[i]['path']:
        #obtengo el path y el contenido del archivo
        get_contenido=requests.get(url + tree[i]['path'], auth=(user, key))
        contenido=get_contenido.json()
        archivos.append((contenido['name'], contenido['content']))
        time.sleep(1)
print (archivos)

In [ ]:
archivos.sort()

import base64
frase=''
for texto in archivos:
    frase+=str(base64.b64decode(texto[1]))

#La frase buscada es:
frase=frase.replace('b\'', '').replace('\\n\'', ' ')
frase

# Scavengers cha-3 V2

In [ ]:
def get_auth(url, headers={'Authorization':'token {}'.format(key)}):
    baseURL='https://api.github.com{}'.format(url)
    res=requests.get(baseURL, headers=headers, auth=(user, key))
    return res.json()

In [ ]:
data=get_auth('/search/code?q=extension:.scavengerhunt repo:ironhack-datalabs/scavenger')

In [ ]:
data['items'][0]['name']

In [ ]:
files=[(e['name'], e['path']) for e in data['items']]
files[2]

In [ ]:
formated=[(e[0], 'https://githubusercontent.com/ironhack-datalabs/scavenger/master/{}'.format(e[1])) for e in files]
formated

In [ ]:
urls=[e[1] for e in sorted(formated, key=lambda x: x[0])]
urls

In [ ]:
lst=[requests.get(e, auth=(user, key)).text for e in urls]
' '.join(lst)

# Web Scrap

In [ ]:
import requests

In [ ]:
url='https://www.reuters.com/article/us-shazam-m-a-apple/apple-completes-deal-to-buy-shazam-idUSKCN1M4202'

In [ ]:
html=requests.get(url).content
p=html[:600].decode()
p.encode()

In [ ]:
!pip3 install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
soup=bs(html, 'lxml')
soup

In [ ]:
tags=['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'p']

text=[e.text for e in soup.find_all(tags)]
text

In [ ]:
'\n'.join(text)

In [ ]:
url='https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy'

In [ ]:
html=requests.get(url).content
soup=bs(html, 'lxml')

In [ ]:
table=soup.find_all('table', {'class':'sortable wikitable'})[0]
table

In [ ]:
rows=table.find_all('tr')
rows=[r.text.strip().split('\n') for r in rows]

final=[]
for e in rows:
    tmp=[]
    for st in e:
        if st!='':
            tmp.append(st)
    final.append(tmp)
final

In [ ]:
import pandas as pd

col_names=final[0]
data=final[1:]
df=pd.DataFrame(data, columns=col_names)
df

In [ ]:
# otra wiki
url='https://es.wikipedia.org/wiki/Juegos_Ol%C3%ADmpicos_de_Barcelona_1992'
res=requests.get(url)

In [ ]:
data=res.text
soup=bs(data, 'html.parser')
soup

In [ ]:
tables=soup.find_all('table')
ultima=tables[-1]
ultima

In [ ]:
elem=ultima.find_all('a')[0]
elem.text

In [ ]:
elem.attrs

In [ ]:
elem.attrs['href']

In [ ]:
medallero=soup.find_all('table')[-4]
medallero

In [ ]:
med_paises=[]

for f in medallero.find_all('tr'):
    fila=[e for e in f.find_all('td')]
    if len(fila)>0:
        pais={'nombre': fila[1].find('a').text.strip(),
              'oros':int(fila[2].text),
              'platas':int(fila[3].text),
              'bronces':int(fila[4].text)}
        med_paises.append(pais)
        
med_paises

In [ ]:
sum([p['oros'] for p in med_paises])

In [ ]:
df=pd.DataFrame(med_paises)
df

In [ ]:
res=requests.get('https://www.emsc-csem.org/Earthquake/')
res.status_code

In [ ]:
soup=bs(res.text.encode('utf-8'))
soup

In [ ]:
import unidecode

t=soup.select('#tbody tr')
m=t[0].select('td')
m

In [ ]:
earthquake={'date':m[3].find('a').text,
            'region':m[-2].text,
            'lat':m[4].text+m[5].text,
            'lng':m[6].text+m[7].text}

earthquake={k:unidecode.unidecode(v).strip() for k,v in earthquake.items()}

type(earthquake)

In [ ]:
df=pd.DataFrame([earthquake])
df

In [ ]:
t=soup.select('#tbody tr')
print (len(t))
earth=[]

for i in range(len(t)):
    m=t[i].select('td')


    earthquake={'date':m[3].find('a').text,
                'region':m[-2].text,
                'lat':m[4].text+m[5].text,
                'lng':m[6].text+m[7].text}

    earthquake={k:unidecode.unidecode(v).strip() for k,v in earthquake.items()}
    earth.append(earthquake)
    
    
df=pd.DataFrame(earth)
df


In [ ]:
# crypto

url='https://es.wikipedia.org/wiki/Criptomoneda'

In [ ]:
res=requests.get(url)
res.text

In [ ]:
soup=bs(res.text, 'html.parser')
soup.title.text

In [ ]:
soup.find_all('td')

In [ ]:
lst_elem=soup.select('tr td a')
lst_elem[0]

In [ ]:
for e in lst_elem:
    print (e.text, e.attrs['href'])

In [ ]:
sel=soup.select('table:nth-of-type(1) tr')[1:-2]
list(map(lambda x:x.findChildren()[0].text.rstrip(), sel))